In [1]:
import AdventOfCode
import Data.List.Split (splitOn)
import Data.List.Split (chunksOf)
import Data.List (foldl', foldl1')
import Data.Bits (xor)
import Data.Char (ord)
import Numeric (showHex)
import qualified Data.Graph.Inductive as G
import qualified Data.Vector.Unboxed as U
import qualified Data.Map.Strict as M
import qualified Data.IntMap.Strict as I
import qualified Data.IntSet as S

input <- dayString 14

In [2]:
circle = U.fromList [0..255]

subIndices vector index len 
    | (index + len) <= U.length vector = let
        indices = [index..(index+len-1)]
        in indices
    | (index + len) > U.length vector = let
        extra = (index+len) - U.length vector
        tillEnd = [index..(U.length vector - 1)]
        fromStart = [0..(extra-1)]
        indices = tillEnd ++ fromStart
        in indices

mutate vector index len = let
    indices   = U.fromList $ subIndices vector index len
    subvector = U.backpermute vector indices
    reversed  = U.reverse subvector
    updated   = U.update vector $ U.zip indices reversed
    in updated

loop orig 0 []     _     _          vector = vector
loop orig n []     index skipLength vector = loop orig (n-1) orig index skipLength vector
loop orig n (l:ls) index skipLength vector = let
    vector' = mutate vector index l
    index' = (index + l + skipLength) `mod` U.length vector
    skipLength' = skipLength + 1
    in loop orig n ls index' skipLength' vector'

hash str = let
    suffixed = map ord str ++ [17, 31, 73, 47, 23]
    looped = loop suffixed 63 suffixed 0 0 circle
    chunked :: [[Int]]
    chunked = chunksOf 16 $ U.toList looped
    hexed = map (flip showHex "" . foldl1' xor) chunked
    in concatMap (\h -> if length h == 1 then '0':h else h) hexed

In [3]:
toBinary = (M.!) $ M.fromList
    [ ('0', "0000")
    , ('1', "0001")
    , ('2', "0010")
    , ('3', "0011")
    , ('4', "0100")
    , ('5', "0101")
    , ('6', "0110")
    , ('7', "0111")
    , ('8', "1000")
    , ('9', "1001")
    , ('a', "1010")
    , ('b', "1011")
    , ('c', "1100")
    , ('d', "1101")
    , ('e', "1110")
    , ('f', "1111")
    ]

toBitString = concatMap toBinary

In [4]:
rows str = map (\i -> str ++ '-':(show i)) [0..127]

bitStrings = map (toBitString . hash) . rows

sum $ map (length . filter (=='1')) (bitStrings input)

8250

In [5]:
bitmap = U.fromList $ concat $ bitStrings input

In [6]:
neighbours bound (x,y) = [b*bound+a | (a,b) <-[(x, y-1), (x-1, y), (x+1, y), (x, y+1)], a >= 0, b >= 0, a < bound, b < bound]

In [7]:
setIndices = U.filter (\(b, i) -> b == '1') $ U.zip bitmap $ U.fromList [0..(128*128)]

fromIndex bound i = let (y,x) = i `divMod` bound in (x,y)

setNeighbours bound graph i = let
    coord = (fromIndex bound i)
    neighbourCoords = neighbours bound coord
    ns = map (graph U.!) neighbourCoords
    bits = zip ns neighbourCoords
    set = filter (\(b, c) -> b == '1') bits
    in map snd set

In [8]:
graph = map (\(_, i) -> (i, setNeighbours 128 bitmap i)) $ U.toList setIndices

In [9]:
edges = concatMap (\(n, es) -> zip (repeat n) es) graph
nodes = map fst graph

In [10]:
length $ G.components (G.mkUGraph nodes edges :: G.Gr () ())

1113